In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import json

club_info = {}

# First code to scrape URLs
r = requests.get("https://en.wikipedia.org/wiki/List_of_Premier_League_clubs")
soup = bs(r.content)

for row in soup.select(".wikitable tbody tr"):
    club_name_element = row.select_one("td a[title^='']")
    if club_name_element:
        club_name = club_name_element.get_text(strip=True)
        club_link = club_name_element['href']

        club_info[club_name] = club_link

# List to store all club data
all_club_data = []

# Second code to scrape infobox using the URLs
for club_name, club_link in club_info.items():
    url = f"https://en.wikipedia.org{club_link}"
    r = requests.get(url)
    soup = bs(r.content)

    info_rows = soup.select(".infobox tbody tr")
    club_data = {}

    for row in info_rows:
        th_element = row.find("th", class_="infobox-label")
        td_element = row.find_next("td", class_="infobox-data")

        if th_element and td_element:
            key = th_element.get_text(strip=True)

            # Check if the key is "Nickname(s)" or "Owners"
            if key == 'Nickname(s)' or key == 'Owners':
                items = list(td_element.stripped_strings)
                club_data[key] = items
            else:
                value = td_element.get_text(strip=True)
                if key == 'Founded':
                    match = re.search(r'\b(\d{4})\b', value)
                    if match:
                        club_data[key] = match.group(1)
                else:
                    club_data[key] = value

    # Append club data to the list
    all_club_data.append({"Club": club_name, "Data": club_data})

# Save the list to a JSON file
with open("PremierL_data.json", "w") as json_file:
    json.dump(all_club_data, json_file, indent=2)